In [1]:
import ee
import geemap
import os

In [2]:
Map = geemap.Map(center=[10.32, 37.75], zoom=10)
Map

Map(center=[10.32, 37.75], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [3]:

work_dir = os.path.expanduser("E:/Researchs/EOTC church forests/Analysis/Codes")

In [35]:
# Import Points to the map
chemoga = geemap.shp_to_ee(os.path.join(work_dir, "DSM_RS.shp"))
Map.addLayer(chemoga, {}, 'Chemoga Watershed')

In [36]:
# Set the Dates 
startDate = '2023-12-01'
endDate = '2024-04-30'

# Import Points to the map
points_2024 = geemap.shp_to_ee(os.path.join(work_dir, "points_2024.shp"))
Map.addLayer(points_2024, {}, 'Points_2024')

In [37]:
def mask_s2_clouds(image):
    
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(startDate, endDate)
    .filterBounds(chemoga)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
    .map(mask_s2_clouds)
)

# Print the number satellite images
print('Number of images: ', dataset.size().getInfo())

Number of images:  9


In [38]:
s2median = dataset.median().clip(chemoga)

In [39]:
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Add the median image to the map
Map.addLayer(s2median, visualization, 'S2 Median Image')

In [40]:
# Display the input imagery and the region in which to do the PCA.
region = chemoga.geometry()
image = s2median.select('B2', 'B3', 'B4', 'B8', 'B9', 'B11', 'B12')

# Set some information about the input to be used later.
scale = 10
bandNames = image.bandNames()

# Mean center the data to enable a faster covariance reducer
# and an SD stretch of the principal components.
meanDict = image.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e9
})
means = ee.Image.constant(meanDict.values(bandNames))
centered = image.subtract(means)

# This helper function returns a list of new band names.
def getNewBandNames(prefix):
  seq = ee.List.sequence(1, bandNames.length())
  return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))


# This function accepts mean centered imagery, a scale and
# a region in which to perform the analysis.  It returns the
# Principal Components (PC) in the region as a new image.
def getPrincipalComponents(centered, scale, region):
  # Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar= arrays.reduceRegion(**{
    'reducer': ee.Reducer.centeredCovariance(),
    'geometry': region,
    'scale': scale,
    'maxPixels': 1e9
  })

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()

  # This is a P-length vector of Eigenvalues.
  eigenValues = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigenVectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt()) \
    .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

  # Turn the PCs into a P-band image, normalized by SD.
  return principalComponents \
    .arrayProject([0]) \
    .arrayFlatten([getNewBandNames('pc')]) \
    .divide(sdImage) \

# Get the PCs at the specified scale and in the specified region
pcImage = getPrincipalComponents(centered, scale, region)

print(pcImage.getInfo())

{'type': 'Image', 'bands': [{'id': 'pc1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc3', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc4', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc5', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc6', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'pc7', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [41]:
# Composite 
composite = ee.Image.cat([(s2median), (pcImage)])

In [42]:
# Import Points to the map
points_2024 = geemap.shp_to_ee(os.path.join(work_dir, "points_2024.shp"))
Map.addLayer(points_2024, {}, 'Points_2024')

# Adds a column of deterministic pseudorandom numbers.
sample_2024 = points_2024.randomColumn("random", 1234)

split = 0.7
trainingGcp_2024 = sample_2024.filter(ee.Filter.lt("random", split))
validationGcp_2024 = sample_2024.filter(ee.Filter.gte("random", split))

print('Training Size:', trainingGcp_2024.size().getInfo())
print('Validation Size:', validationGcp_2024.size().getInfo())

# Train the classifier 
bands = ['B2', 'B3', 'B4', 'B8', 'B9', 'B11', 'B12', 'pc1', 'pc2', 'pc3']
            
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = composite.sampleRegions(
    collection=trainingGcp_2024, 
    properties=[label], 
    scale=30
)


Training Size: 924
Validation Size: 439


In [43]:
RF_classifier = ee.Classifier.smileRandomForest(60).train(sample, label, bands);

# Classify the image with the same bands used for training.
RF_2024 = composite.classify(RF_classifier)

# Accuracy Assessment 
band = 'classification'

test = RF_2024.select(band).sampleRegions(
    collection=validationGcp_2024, 
    properties=[label],
    scale=30
)

test_accuracy = test.errorMatrix('landcover', 'classification')
print('RF error matrix: ', test_accuracy.getInfo())
print('RF overall accuracy: ', test_accuracy.accuracy().getInfo())
print('RF kappa coefficient: ', test_accuracy.kappa().getInfo())
print('RF producer accuracy: ', test_accuracy.producersAccuracy().getInfo())
print('RF consumer accuracy: ', test_accuracy.consumersAccuracy().getInfo())

RF error matrix:  [[0, 0, 0, 0, 0, 0], [0, 47, 1, 0, 0, 2], [0, 2, 5, 0, 0, 0], [0, 1, 0, 3, 0, 0], [0, 0, 0, 0, 7, 1], [0, 1, 0, 0, 0, 36]]
RF overall accuracy:  0.9245283018867925
RF kappa coefficient:  0.8811159399971962
RF producer accuracy:  [[0], [0.94], [0.7142857142857143], [0.75], [0.875], [0.972972972972973]]
RF consumer accuracy:  [[0, 0.9215686274509803, 0.8333333333333334, 1, 1, 0.9230769230769231]]


In [44]:
GTB_classifier = ee.Classifier.smileGradientTreeBoost(60).train(sample, label, bands);

# Classify the image with the same bands used for training.
GTB_2024 = composite.classify(GTB_classifier)

# Accuracy Assessment 
band = 'classification'

test = GTB_2024.select(band).sampleRegions(
    collection=validationGcp_2024, 
    properties=[label],
    scale=30
)

test_accuracy = test.errorMatrix('landcover', 'classification')
print('GTB error matrix: ', test_accuracy.getInfo())
print('GTB overall accuracy: ', test_accuracy.accuracy().getInfo())
print('GTB kappa coefficient: ', test_accuracy.kappa().getInfo())
print('GTB producer accuracy: ', test_accuracy.producersAccuracy().getInfo())
print('GTB consumer accuracy: ', test_accuracy.consumersAccuracy().getInfo())

GTB error matrix:  [[0, 0, 0, 0, 0, 0, 0], [0, 47, 1, 0, 0, 2, 0], [0, 2, 4, 0, 0, 0, 1], [0, 1, 0, 3, 0, 0, 0], [0, 0, 0, 0, 7, 1, 0], [0, 0, 0, 0, 0, 37, 0], [0, 0, 0, 0, 0, 0, 0]]
GTB overall accuracy:  0.9245283018867925
GTB kappa coefficient:  0.8814483433524396
GTB producer accuracy:  [[0], [0.94], [0.5714285714285714], [0.75], [0.875], [1], [0]]
GTB consumer accuracy:  [[0, 0.94, 0.8, 1, 1, 0.925, 0]]


In [31]:
SVM_classifier = ee.Classifier.libsvm(kernelType='LINEAR', cost=10).train(sample, label, bands);

# Classify the image with the same bands used for training.
SVM_2024 = composite.classify(SVM_classifier)

# Accuracy Assessment 
band = 'classification'

test = SVM_2024.select(band).sampleRegions(
    collection=validationGcp_2024, 
    properties=[label],
    scale=30
)

test_accuracy = test.errorMatrix('landcover', 'classification')
print('SVM error matrix: ', test_accuracy.getInfo())
print('SVM overall accuracy: ', test_accuracy.accuracy().getInfo())
print('SVM kappa coefficient: ', test_accuracy.kappa().getInfo())
print('SVM producer accuracy: ', test_accuracy.producersAccuracy().getInfo())
print('SVM consumer accuracy: ', test_accuracy.consumersAccuracy().getInfo())

SVM error matrix:  [[0, 0, 0, 0, 0, 0], [0, 47, 1, 0, 0, 2], [0, 3, 4, 0, 0, 0], [0, 2, 0, 2, 0, 0], [0, 0, 0, 0, 6, 2], [0, 2, 0, 0, 0, 35]]
SVM overall accuracy:  0.8867924528301887
SVM kappa coefficient:  0.818337617823479
SVM producer accuracy:  [[0], [0.94], [0.5714285714285714], [0.5], [0.75], [0.9459459459459459]]
SVM consumer accuracy:  [[0, 0.8703703703703703, 0.8, 1, 1, 0.8974358974358975]]


In [ ]:
# Visualize the classification
Map.addLayer(
    RF_2024,
    {
        'min': 1,
        'max': 6,
        'palette': [
            '006400', '8B4513', 'FFFF00', '00FF00', '0000FF', 'FF0000'
        ]
    },
    'RF Classification'
)
Map

In [15]:
# Agriculture Area
agriculture = RF_2024.eq(1).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_ag = agriculture.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Area of Agriculture:', round(area_ag.getInfo()['classification']))

# Bareland Area
bareland = RF_2024.eq(2).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_br = bareland.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Area of Bareland:', round(area_br.getInfo()['classification']))

# Built-up Area
builtup = RF_2024.eq(3).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_bu = builtup.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Area of Built-up:', round(area_bu.getInfo()['classification']))

# Forest Area
forest = RF_2024.eq(4).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_fr = forest.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Area of Forest:', round(area_fr.getInfo()['classification']))

# Grazing Area
grazing = RF_2024.eq(5).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_gr = grazing.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Area of Grazing:', round(area_gr.getInfo()['classification']))

# Waterbody Area
water = RF_2024.eq(6).multiply(ee.Image.pixelArea().divide(10000))

# Reduce the region to calculate the sum of the areas that meet the condition.
area_wb = water.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': chemoga.geometry(),  # Assuming jedeb is a predefined geometry.
  'scale': 10,
  'maxPixels': 1e10
})

print('Water area:', round(area_wb.getInfo()['classification']))

Area of Built-up: 156
Area of Forest: 1813
Area of Grazing: 7068
Water area: 18


In [ ]:
# Export the classified image to local drive
geemap.ee_export_image(RF_2024, filename='RF_Chemoga_2024_Sime.tif', scale=10, region=chemoga.geometry(), file_per_band=False)